Цель: подготовить данные к обучению модели и сформировать таргет

In [1]:
from pathlib import Path
import re

import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import word_tokenize
from pymorphy3 import MorphAnalyzer

Загрузка данных

In [2]:
RAW_PATH = Path("../data/raw/data_base.csv")
SAMPLE_PATH = Path("../data/sample/sample.csv")
OUT_DIR = Path("../data/processed")

In [3]:
P = 0.90 # квантиль для формирования целевого класса
SEED = 42

In [4]:
if RAW_PATH.exists():
    in_path = RAW_PATH
elif SAMPLE_PATH.exists():
    in_path = SAMPLE_PATH
else:
    raise FileNotFoundError("Нет ничего")

In [5]:
print("[preprocess] input:", in_path)

[preprocess] input: ..\data\raw\data_base.csv


In [6]:
df = pd.read_csv(in_path)

Оценка размера датасета, типов столбцов и доли пропусков, чтобы оценить объём предобработки

In [7]:
df.head(3)

,id_post,date,text,views,reactions,comments
0,14381,2026-02-02 13:31:57+00:00,Меню «Пуск» в Windows 11 [превратили](https://...,74536,1064,0
1,14380,2026-02-02 10:49:42+00:00,В Москве робота заметили за уборкой снега. \n\...,110566,1442,0
2,14376,2026-02-02 10:01:14+00:00,В Steam вышел хоррор The 18th Attic — в нем ну...,104438,675,0


In [8]:
print("Shape:", df.shape)
display(df.info())
df.isna().mean().sort_values(ascending=False).head(10)

Shape: (3351, 6)
<class 'pandas.DataFrame'>
RangeIndex: 3351 entries, 0 to 3350
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   id_post    3351 non-null   int64
 1   date       3351 non-null   str  
 2   text       3351 non-null   str  
 3   views      3351 non-null   int64
 4   reactions  3351 non-null   int64
 5   comments   3351 non-null   int64
dtypes: int64(4), str(2)
memory usage: 157.2 KB


None

id_post      0.0
date         0.0
text         0.0
views        0.0
reactions    0.0
comments     0.0
dtype: float64

In [9]:
df0 = df.copy()

In [10]:
(df0["comments"] > 0).sum(), len(df0)


(np.int64(0), 3351)

In [11]:
df0.loc[df0["comments"] > 0, ["id_post", "date", "views", "reactions", "comments", "text"]].head(20)


,id_post,date,views,reactions,comments,text


comments не содержит ненулевых значений, поэтому столбец будет удалён как неинформативный

Привожу столбцы к корректным типам и удаляю строки, которые не подходят для анализа:
- некорректная дата
- views ≤ 0
- отрицательные реакции

In [12]:
if (df0["comments"].fillna(0) > 0).sum() == 0:
    df0 = df0.drop(columns=["comments"])

In [13]:
df0["date"] = pd.to_datetime(df0["date"], errors="coerce", utc=True)
for c in ["views", "reactions"]:
    df0[c] = pd.to_numeric(df0[c], errors="coerce")

In [14]:
before = len(df0)
df0 = df0.dropna(subset=["date"])
df0 = df0[df0["views"].fillna(0) > 0]
df0 = df0[df0["reactions"].fillna(0) >= 0]
after = len(df0)

In [15]:
before, after, before-after

(3351, 3351, 0)

Определяю метрику вовлечённости: engagement = reactions / views  
Целевой класс target=1 задаю как верхние 10% постов по engagement (P=0.90)

In [16]:
df0["engagement"] = df0["reactions"].fillna(0) / df0["views"]

In [17]:
thr = df0["engagement"].quantile(P)
df0["target"] = (df0["engagement"] >= thr).astype(int)

In [18]:
print(f"P={P}, threshold={thr:.6f}")
df0["target"].value_counts()

P=0.9, threshold=0.012574


target
0    3015
1     336
Name: count, dtype: int64

**Результат:** рассчитан порог (90-й перцентиль) и получено распределение классов target

In [19]:
df0["engagement"].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.97, 0.99])

count    3351.000000
mean        0.007431
std         0.006488
min         0.000000
50%         0.006440
75%         0.009592
90%         0.012574
95%         0.014752
97%         0.016730
99%         0.020276
max         0.302003
Name: engagement, dtype: float64

Добавляю признаки времени публикации: weekday, hour, month, is_weekend.  
Они могут влиять на вовлечённость из-за активности посетителей канала в разные дни/часы

In [20]:
df0["weekday"] = df0["date"].dt.dayofweek
df0["hour"] = df0["date"].dt.hour
df0["month"] = df0["date"].dt.month
df0["is_weekend"] = df0["weekday"].isin([5, 6]).astype(int)

In [21]:
df0[["date","weekday","hour","is_weekend"]].head()

,date,weekday,hour,is_weekend
0,2026-02-02 13:31:57+00:00,0,13,0
1,2026-02-02 10:49:42+00:00,0,10,0
2,2026-02-02 10:01:14+00:00,0,10,0
3,2026-02-02 07:14:58+00:00,0,7,0
4,2026-02-01 14:05:02+00:00,6,14,1


Подгружаю ресурсы NLTK для токенизации и списка стоп-слов

In [22]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Arina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Arina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Arina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [23]:
stopwords = set(nltk.corpus.stopwords.words("russian"))
morph = MorphAnalyzer()

In [24]:
def has_duplicate_characters(word):
    return re.search(r"(.)\1{2,}", word) is not None

Очищаю текст и привожу его к нормализованному виду:
- удаление ссылок/упоминаний/email
- удаление символов, которые не являются буквами
- токенизация
- удаление стоп-слов
- лемматизация (pymorphy3)

In [25]:
def preprocess_text(text):
    if pd.isna(text):
        return ""

    s = str(text).lower()

    # ссылки/упоминания/почты
    s = re.sub(r"\[.*?\]\(https?:\/\/[^\s)]+\)", " ", s)
    s = re.sub(r"http\S+|www\S+|https\S+", " ", s)
    s = re.sub(r"tg://user\?id=\d+", " ", s)
    s = re.sub(r"@\w+", " ", s)
    s = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", " ", s)

    # только буквы
    s = re.sub(r"[^a-zа-яё]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    if not s:
        return ""

    tokens = word_tokenize(s, language="russian")
    tokens = [t for t in tokens if len(t) > 1]
    tokens = [t for t in tokens if t not in stopwords]
    tokens = [t for t in tokens if not has_duplicate_characters(t)]
    if not tokens:
        return ""

    lemmas = [morph.parse(t)[0].normal_form for t in tokens]
    return " ".join(lemmas)

In [26]:
df0["processed_text"] = df0["text"].fillna("").apply(preprocess_text)

In [27]:
df0["text_len"] = df0["processed_text"].str.len()
df0["words_cnt"] = df0["processed_text"].str.split().map(len)

In [28]:
demo = df0[["id_post","text","processed_text","target"]].head(5).copy()
demo["text"] = demo["text"].str.replace("\n"," ").str.slice(0,120) + "..."
demo["processed_text"] = demo["processed_text"].str.slice(0,120) + "..."

Пример до/после

In [29]:
demo

,id_post,text,processed_text,target
0,14381,Меню «Пуск» в Windows 11 [превратили](https://...,меню пуск windows ios microsoft начать раскаты...,1
1,14380,В Москве робота заметили за уборкой снега. В...,москва робот заметить уборка снег восстание ма...,1
2,14376,В Steam вышел хоррор The 18th Attic — в нем ну...,steam выйти хоррор the th attic немой нужно гл...,0
3,14374,Бот Openclaw (он же Clawdbot) сливает данные с...,бот openclaw clawdbot сливать дать свой пользо...,0
4,14373,⚡️ Разыгрываем MacBook Pro M5 и поход в баню с...,разыгрывать macbook pro поход баня админ канал...,0


Делю данные на обучающую/валидационную/тестовую выборки со стратификацией по target, чтобы сохранить долю positive-класса во всех частях.

In [30]:
train, test = train_test_split(df0, test_size=0.15, random_state=SEED, stratify=df0["target"])
train, val = train_test_split(train, test_size=0.15, random_state=SEED, stratify=train["target"])

In [31]:
len(train), len(val), len(test), train["target"].mean(), val["target"].mean(), test["target"].mean()

(2420,
 428,
 503,
 np.float64(0.10041322314049586),
 np.float64(0.10046728971962617),
 np.float64(0.09940357852882704))

In [32]:
OUT_DIR.mkdir(parents=True, exist_ok=True)

df0.to_csv(OUT_DIR / "data_processed.csv", index=False)
train.to_csv(OUT_DIR / "data_train.csv", index=False)
val.to_csv(OUT_DIR / "data_val.csv", index=False)
test.to_csv(OUT_DIR / "data_test.csv", index=False)

In [33]:
print("Saved to:", OUT_DIR.resolve())

Saved to: C:\Users\Arina\VS Code Projects\social-media-analytics\data\processed
